Randomization inference is probably the way we want to go in order to handle very skewed distributions of continuous metrics.  We need to be able to supply the following functionality:

1. Generic randomization inference for any scalar test statistic

2. A selection of pre-built test statistics along with documentation about what they mean and when you might want to use them

3. A method for doing hypothetical assignment, or supplying your own function (we may want to scale this up at some point to have some pre-built functions). We'll focus on binary assignment for now (i.e. only two variants), and we'll work to generalize this later.

4. Power analysis. This is relatively easy, but we need a way to do run time estimations as well

In [3]:
import os
import sys
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy 

from matplotlib import style
from tqdm.auto import tqdm

In [40]:
# Additional imports 
from functools import partial

In [4]:
# Set pandas preferences
pd.options.display.max_columns=500
pd.options.display.max_colwidth=500
pd.options.display.max_rows=500

In [5]:
# Set plot style
style.use("fivethirtyeight")

In [6]:
# Initialize tqdm for pandas
tqdm.pandas()

In [7]:
# Example dataset: from https://www.franciscoyira.com/post/randomization-inference-causal-mixtape/

In [8]:
name_list = ['Andy', 'Ben', 'Chad', 'Daniel', 'Edith', 'Frank', 'George', 'Hank']
d_list = [1,1,1,1,0,0,0,0]
y_list = [10,5,16,3,5,7,8,10]
y0_list = [np.nan, np.nan, np.nan, np.nan, 5, 7, 8, 10]
y1_list = [10,5,16,3,np.nan,np.nan,np.nan,np.nan]
id_unit_list = [1,2,3,4,5,6,7,8]

In [9]:
df = pd.DataFrame()
df['name'] = name_list
df['d'] = d_list
df['y'] = y_list
df['y0'] = y0_list
df['y1'] = y1_list
df['id_unit'] = id_unit_list

In [10]:
df

,name,d,y,y0,y1,id_unit
0,Andy,1,10,NaN,10.0,1
1,Ben,1,5,NaN,5.0,2
2,Chad,1,16,NaN,16.0,3
3,Daniel,1,3,NaN,3.0,4
4,Edith,0,5,5.0,NaN,5
5,Frank,0,7,7.0,NaN,6
6,George,0,8,8.0,NaN,7
7,Hank,0,10,10.0,NaN,8


In [26]:
t = "d"
val = 1

In [30]:
df.query("d==1")

,name,d,y,y0,y1,id_unit
0,Andy,1,10,NaN,10.0,1
1,Ben,1,5,NaN,5.0,2
2,Chad,1,16,NaN,16.0,3
3,Daniel,1,3,NaN,3.0,4


In [31]:
df.query("{0}==@val".format(t))

,name,d,y,y0,y1,id_unit
0,Andy,1,10,NaN,10.0,1
1,Ben,1,5,NaN,5.0,2
2,Chad,1,16,NaN,16.0,3
3,Daniel,1,3,NaN,3.0,4


In [196]:
def ri_test_statistic_difference_in_means(df, outcome_col, treatment_col, treatment_name, control_name):
    
    sdo = df.query("{0}==@treatment_name".format(treatment_col))[outcome_col].mean(numeric_only=True) - df.query("{0}==@control_name".format(treatment_col))[outcome_col].mean(numeric_only=True)

    return sdo

In [186]:
def ri_test_statistic_difference_in_ks(df, outcome_col, treatment_col, treatment_name, control_name, alternative='two-sided'):
    
    ks_ = stats.ks_2samp(df.query("{0}==@treatment_name".format(treatment_col))[outcome_col].values - df.query("{0}==@control_name".format(treatment_col))[outcome_col].values, alternative=alternative)
    
    return ks_

In [187]:
def ri_test_statistic_difference_in_quantiles(df, outcome_col, treatment_col, treatment_name, control_name, quantile=0.5):
    
    q_diff = df.query("{0}==@treatment_name".format(treatment_col))[outcome_col].quantile(q=quantile) - df.query("{0}==@control_name".format(treatment_col))[outcome_col].quantile(q=quantile)
    
    return q_diff

The steps here for analyzing experimental data are:

1. Choose a sharp null

2. Pick a test statistic

3. Select randomization procedure 
    
    a. Run the permutations

4. Get the p-values by comparing to the true statistic with the simulated distribution

5. Calculate confidence intervals

For power analysis:

1. Choose a sharp null

2. Pick a test statistic

3. Select the desired power and confidence level

4. compute mme

We'll need to run sample size estimation as well, so probably we'll need to sample the historical data and run calculations from there. Then tie the sample sizes back to run times

In [236]:
class RandomizationInference:
    
    def __init__(self):
        
        self._supported_test_statistics = {'difference_in_means': ri_test_statistic_difference_in_means, 
                              'difference_in_percentiles': ri_test_statistic_difference_in_quantiles, 
                              'difference_in_ks_statistic': ri_test_statistic_difference_in_ks}
        self.df_sims = None
        self.observed_test_satistic = None
        self.p_val = None
        self.ci = None
        # self.df = df
        # self.sharp_null = sharp_null
        # self.test_statistic = test_statistic
        # self.outcome_column_name = outcome_column_name
        # self.treatment_column_name = treatment_column_name
        # self.treatment_name = treatment_name
        # self.control_name = control_name
        # self.num_permutations = num_permutations
        
    @staticmethod
    def sharp_null(df_, sharp_null_type, sharp_null_value, outcome_column_name):
        
        if sharp_null_type == 'additive':
            df_['outcome_sharp_null'] = df_[outcome_column_name] + sharp_null_value
        else:
            df_['outcome_sharp_null'] = df_[outcome_column_name] * df_[outcome_column_name] * sharp_null_value
        
        return df_
    
    
    def select_test_statistic(self, test_statistic: dict):
        
        if type(test_statistic['function']) == str:
            assert test_statistic['function'] in self._supported_test_statistics, "test statistic {0} is not currently support. Please select from {1} for implement your own function".format(test_statstic['function'], self._supported_test_statistics)
            test_statistic['function'] = self._supported_test_statistics[test_statistic['function']]
        else:
            assert callable(test_statistic['function']), "supplied custom test statistic {0} is neither a supported type nor a function".format(test_statstic['function'])
        
        if test_statistic['params'] is None:
            test_statistic['params'] = {}
        else:
            assert type(test_statistic['params']) == dict, "Please pass a dictionary of parameters to your test statistic function. The keys should be the parameter names and the values the parameter values."
        
        test_statistic_function = partial(test_statistic['function'], **test_statistic['params'])
        
        return test_statistic_function

    
    @staticmethod
    def make_hypothetical_assignments(df_, treatment_assignment_probability):
        
        assert treatment_assignment_probability > 0 and treatment_assignment_probability < 1, "Treatment assignment probabilities must be great than 0 and less than 1. Received {0}".format(treatment_assignment_probability)
        
        # set the random seed
        np.random.seed()
        
        df_['ri_in_treatment'] = stats.binom.rvs(n=1, p=treatment_assignment_probability, size=df_.shape[0])
        
        return df_
    
    
    def p_value(self, alternative):
        
       #  df_ = self.df_sims.copy()
        
        if not self.observed_test_satistic in self.df_sims['test_statistic'].values:
            add_row = pd.DataFrame({"permutation": [-1], "test_statistic": self.observed_test_satistic})
            self.df_sims = pd.concat([add_row, self.df_sims])
            observed_perm = -1
        else:
            observed_perm = self.df_sims.loc[(self.df_sims['test_statistic'] == self.observed_test_satistic)]['permutation'].values[0]
        
        if alternative == 'two-sided':
            self.df_sims['rank_column'] = np.abs(self.df_sims['test_statistic'])
        else:
            self.df_sims['rank_column'] = self.df_sims['test_statistic']
        
        self.df_sims['rank'] = self.df_sims['rank_column'].rank(method='max', ascending=False)
        
        p_value = self.df_sims.query("permutation==@observed_perm")['rank'].values[0] / self.df_sims.shape[0]
        
        return p_value
    
    
    def get_ci(self, confidence):

        ci_low = 100 * (1 - confidence) / 2
        ci_high = 100 * confidence + ci_low
        null_ci = np.percentile(self.df_sims['test_statistic'].values, [ci_low, ci_high])
        ci_ = null_ci + self.observed_test_satistic
    
        return ci_
    
    
    def run_randomization_inference(self, df, test_statistic_function, treatment_assignment_probability):
        
        df_ = self.make_hypothetical_assignments(df_=df, treatment_assignment_probability=treatment_assignment_probability)
        
        stat_ = test_statistic_function(df=df_, outcome_col='outcome_sharp_null', treatment_col='ri_in_treatment', treatment_name=1, control_name=0)
        
        return stat_
    
    
    def experimental_analysis(self, df, sharp_null_type='additive', sharp_null_value=0, test_statistic={'function': 'difference_in_means', 'params': None}, treatment_assignment_probability=0.5, outcome_column_name='y', treatment_column_name='d', treatment_name=1, control_name=0, num_permutations=1000, alternative='two-sided', confidence=0.95):
        
        assert sharp_null_type in ['additive', 'multiplicative'], "only additive or multiplicative sharp nulls are supported. Received {0}".format(sharp_null_type)
        
        assert type(num_permutations) == int, "Only an interger number of permutations is possible. Received {0}".format(num_permutations)
        
        assert alternative in ['two-sided', 'less', 'greater'], "Only {0} alternatives are supported. Received {0}".format(alternative)
        
        # Copy the input DataFrame so that we don't modify the original data in_place
        df_ = df.copy()
        
        # Step 1: implement the selected sharp null
        df_ = self.sharp_null(df_=df_, sharp_null_type=sharp_null_type, sharp_null_value=sharp_null_value, outcome_column_name=outcome_column_name)
        
        # Step 2: pick a test statistic. We have a list of pre-built ones, otherwise a function must be supplied
        # This function must consume a DataFrame and return a single scalar value
        test_statistic_function = self.select_test_statistic(test_statistic=test_statistic)
        
        # Step 3: Select a randomization proceedure. 
        # Currently this only supports random assignment via coin flip. The probability of being in the treatment group is customizable
        # Also run the simulations
        print('Running {0} permutations'.format(num_permutations))
        sim_dict = {i: self.run_randomization_inference(df=df_, test_statistic_function=test_statistic_function, treatment_assignment_probability=treatment_assignment_probability) for i in tqdm(range(num_permutations))}

        self.df_sims = pd.DataFrame.from_dict(sim_dict, orient='index')
        self.df_sims = self.df_sims.reset_index()
        self.df_sims.columns = ['permutation', 'test_statistic']
    
        # Step 4: calculate p-values
        # First, calculate the observed statistic difference
        self.observed_test_satistic = test_statistic_function(df=df_, outcome_col=outcome_column_name, treatment_col=treatment_column_name, treatment_name=treatment_name, control_name=control_name)

        self.p_val = self.p_value(alternative=alternative)
        
        # Step 5: calculate confidence interval. This is the CI of the test statistic under the null
        ci = self.get_ci(confidence=confidence)
        
        # final output. This should be summarized in a plot
        
                

In [237]:
test_run = RandomizationInference()

In [259]:
test_run.experimental_analysis(df=df, 
                               sharp_null_type='additive', 
                               sharp_null_value=0, 
                               test_statistic={'function': 'difference_in_means', 'params': None}, 
                               treatment_assignment_probability=0.5, 
                               outcome_column_name='y', 
                               treatment_column_name='d', 
                               treatment_name=1, 
                               control_name=0, 
                               num_permutations=1000, 
                               alternative='two-sided', 
                               confidence=0.95)

Running 1000 permutations


  0%|          | 0/1000 [00:00<?, ?it/s]

In [260]:
test_run.p_val

0.801

In [261]:
test_run.observed_test_satistic

1.0

In [262]:
test_run.df_sims.head()

,permutation,test_statistic,rank_column,rank
0,0,-4.500000,4.500000,175.0
1,1,-3.733333,3.733333,250.0
2,2,-4.800000,4.800000,135.0
3,3,-1.000000,1.000000,801.0
4,4,2.133333,2.133333,541.0


In [263]:
test_run.df_sims['test_statistic'].nunique()

83

In [264]:
test_run.df_sims['test_statistic'].unique()

array([-4.5       , -3.73333333, -4.8       , -1.        ,  2.13333333,
       -3.42857143,  6.66666667, -2.13333333,  3.73333333, -2.        ,
        0.66666667,  3.33333333,  1.        ,  0.        ,  1.6       ,
       -4.8       ,  1.14285714,  0.53333333, -0.66666667,  4.26666667,
        4.8       ,  2.66666667, -0.53333333,  3.2       ,  9.14285714,
        2.        ,  3.5       ,  5.5       ,  1.06666667, -1.14285714,
       -3.2       , -0.53333333,  4.5       , -5.5       ,  5.71428571,
       -3.        , -3.5       , -4.66666667,  1.06666667, -3.33333333,
       -5.86666667, -2.5       , -0.5       ,  3.73333333, -1.33333333,
        5.33333333, -0.66666667, -1.6       , -1.06666667,  4.        ,
        3.        ,         nan,  3.33333333,  0.66666667, -5.71428571,
        0.5       , -5.33333333,  1.33333333, -4.        , -9.14285714,
       -2.66666667, -1.06666667,  6.        ,  0.53333333, -2.28571429,
       -4.26666667,  4.66666667,  3.42857143,  4.8       ,  5.86

In [252]:
test_run.df_sims['permutation'].min()

0

In [253]:
test_run.df_sims.query("test_statistic==1")

,permutation,test_statistic,rank_column,rank
104,104,1.0,1.0,8056.0
106,106,1.0,1.0,8056.0
146,146,1.0,1.0,8056.0
225,225,1.0,1.0,8056.0
439,439,1.0,1.0,8056.0
470,470,1.0,1.0,8056.0
492,492,1.0,1.0,8056.0
517,517,1.0,1.0,8056.0
532,532,1.0,1.0,8056.0
585,585,1.0,1.0,8056.0


In [256]:
test_run.df_sims.query("test_statistic==-1")

,permutation,test_statistic,rank_column,rank
11,11,-1.0,1.0,8056.0
43,43,-1.0,1.0,8056.0
128,128,-1.0,1.0,8056.0
143,143,-1.0,1.0,8056.0
163,163,-1.0,1.0,8056.0
177,177,-1.0,1.0,8056.0
187,187,-1.0,1.0,8056.0
213,213,-1.0,1.0,8056.0
215,215,-1.0,1.0,8056.0
252,252,-1.0,1.0,8056.0


In [245]:
# So why is this different?
test_run.df_sims['rank'].nunique()

42

In [254]:
 0.8571429 * 70

60.000003

In [257]:
test_run.df_sims['rank_column'].nunique()

42

In [258]:
8056/10000

0.8056